In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas 
import matplotlib.pyplot as plt
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from pymystem3 import Mystem


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



russian_stopwords = stopwords.words("russian")
path = "C:\\Proganiy\\pp-laba3\\annotation.csv"
print(russian_stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [38]:
def create_ann(annatation: str) -> pandas.DataFrame:

    """Создаёт датафрейм по пути аннатации"""

    frame = pandas.DataFrame(columns =["Оценка","Kоличество слов","Текст рецензии"])
    ann_temp = open(annatation, "r", encoding="utf-8")
    for otzv in ann_temp.readlines():
        mas_otzv = otzv.split(",")
        otzv_temp = open(mas_otzv[0],"r",encoding="utf-8")
        otzv_text = " ".join(otzv_temp)
        row = pandas.Series({"Оценка": int(mas_otzv[2]),"Kоличество слов": len(otzv_text), "Текст рецензии": otzv_text})
        new_row = pandas.DataFrame([row], columns=frame.columns)
        frame = pandas.concat([frame, new_row], ignore_index=True)
    frame.dropna()
    return frame

data = create_ann(path)
print(data)

     Оценка Kоличество слов                                     Текст рецензии
0         1            1853  11 марта этого года переводила 145 в Германию(...
1         1            4278  В пятницу, 22 апреля 2022 г. Сбербанк заблокир...
2         1            2035  Здравствуйте. Перед оформлением досудебной пре...
3         1            3503  Я являюсь клиентом Сбербанка уже более десяти ...
4         1            3340  Призываю предпринимателей подумать дважды, пре...
...     ...             ...                                                ...
2995      5             304  Сегодня хочу рассказать о Сбербанке. В этом пр...
2996      5             863  Банк Сбербанк - это прекрасное финансовое учре...
2997      5             979  Пользуюсь сбербанком сколько себя помню, зарпл...
2998      5            2084  Я не сразу научилась пользоваться кредитной сб...
2999      5             777  Пользуюсь сбербанком давно, никаких нарекании ...

[3000 rows x 3 columns]


In [39]:
def clean (frame: pandas.DataFrame) -> pandas.DataFrame:
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(delete_punc)
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(Mystem().lemmatize)
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(lambda x: ' '.join(x))
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(word_tokenize)
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(lambda x: ' '.join(x))
    return frame

def delete_punc(strings: str) -> str:
    punc = '''!()-[]{};:'",<>./?@#$%^&*~1234567890'''
    for char in strings:
        if char in punc:
            strings = strings.replace(char, "")
    return strings

data = clean(data)
print(data)

     Оценка Kоличество слов                                     Текст рецензии
0         1            1853   марта этого года переводила  в Германиюденежн...
1         1            4278  В пятницу  апреля  г Сбербанк заблокировал мою...
2         1            2035  Здравствуйте Перед оформлением досудебной прет...
3         1            3503  Я являюсь клиентом Сбербанка уже более десяти ...
4         1            3340  Призываю предпринимателей подумать дважды преж...
...     ...             ...                                                ...
2995      5             304  Сегодня хочу рассказать о Сбербанке В этом при...
2996      5             863  Банк Сбербанк  это прекрасное финансовое учреж...
2997      5             979  Пользуюсь сбербанком сколько себя помню зарпла...
2998      5            2084  Я не сразу научилась пользоваться кредитной сб...
2999      5             777  Пользуюсь сбербанком давно никаких нарекании н...

[3000 rows x 3 columns]
     Оценка Kоличество слов

In [40]:
max_words = 10000
cv = CountVectorizer(max_features=max_words)
sparse_matrix = cv.fit_transform(data['Текст рецензии']).toarray()
sparse_matrix.shape



(3000, 10000)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(data['Оценка']))
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [60]:


class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 1000)
        self.linear2 = nn.Linear(1000, 100)
        self.linear3 = nn.Linear(100, 5)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

model = LogisticRegression()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)




TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [45]:

epochs = 20
model.train()
loss_values = []
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss_values.append(loss.item())
    pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
    acc = pred * 100.0 / len(x_train)
    print('Epoch: {}, Loss: {}, Accuracy: {}%'.format(epoch+1, loss.item(), acc.numpy()))
    loss.backward()
    optimizer.step()

TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not numpy.ndarray

In [ ]:


model.eval()
with torch.no_grad():
    y_pred = model(x_test)
    loss = criterion(y_pred, y_test)
    pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
    print ("Accuracy : {}%".format(100*pred/len(x_test)))